## Definition of the problem. 

A business owner wants to open a new coffee shop in San Francisco. We will try to solve the problem using the foursquare location data in order to choose the most suitable neighborhood in the city for the purpose. 

The purpose is to recommend a neighborhood in San Francisco which is most suitable for opening a new coffee shop.The right location of the shop is key to it's success.
The solution is suitable to all kind of business owners which operates coffee shops or other different type of restaurants , bars, pubs and look for a location for the new one. 

The proposed solution is to gather the needed data about neighborhoods in the city and the data about all coffee shops in them. After this we will cluster the neighborhoods and we will chose the best one based on a criteria.



For the purpose of the project we will scrap the data about the neighborhoods in San Francisco from a web page. After this we will add the geographic coordinates for each neighborhood. 
After this for each neighborhood will be retrieved the location data from Foursquare in order to cluster the neighborhoods.

The algorithm which will be used is k Means. 



Fist install the needed libraries and packages. 

In [7]:
import pandas as pd
import requests
import folium
import random
!conda install -c conda-forge beautifulsoup4 lxml --yes 
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge geocoder --yes
from bs4 import BeautifulSoup
import json
import numpy as np
import urllib
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import geocoder 
print ("Libraries imported")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported


We get the data of the neighborhoods in San Francisco from a web page using BeautifulSoup.After this we read the data and pass it to a pandas data frame. We look at the 5 five records in our data. 

In [95]:
url = requests.get('http://www.healthysf.org/bdi/outcomes/zipmap.htm')

soup = BeautifulSoup(requests.get('http://www.healthysf.org/bdi/outcomes/zipmap.htm').text, 'lxml')
table = soup.find_all('table')
df = pd.read_html(str(table))
df = pd.DataFrame(df[4])

df.columns = df.iloc[0]
df = df.iloc[1:-1, :-1]
sf_data = df
sf_data.head()

,Zip Code,Neighborhood
1,94102,Hayes Valley/Tenderloin/North of Market
2,94103,South of Market
3,94107,Potrero Hill
4,94108,Chinatown
5,94109,Polk/Russian Hill (Nob Hill)


We retrieve the geo coordinates for each neighborhood based on the zip code. We check the new dataset. 

In [99]:
!pip install uszipcode
from uszipcode import SearchEngine

search = SearchEngine(simple_zipcode=True)

latitude = []
longitude = []

for index, row in sf_data.iterrows():
    zipcode = search.by_zipcode(row["Zip Code"]).to_dict()
    latitude.append(zipcode.get("lat"))
    longitude.append(zipcode.get("lng"))

sf_data["Latitude"] = latitude
sf_data["Longitude"] = longitude

sf_data.head()

,Zip Code,Neighborhood,Latitude,Longitude
1,94102,Hayes Valley/Tenderloin/North of Market,37.780,-122.420
2,94103,South of Market,37.780,-122.410
3,94107,Potrero Hill,37.770,-122.390
4,94108,Chinatown,37.791,-122.409
5,94109,Polk/Russian Hill (Nob Hill),37.790,-122.420


 ## Foursquare data
We create our credential for the Foursquare API and set the location of San Francisco as addres.


In [100]:
from pandas.io.json import json_normalize
CLIENT_ID = 'ZNIGMWEGB3B2PRX1KKCUDR52MK2HBXMHHHXZUKK2D1KJDOIS' # your Foursquare ID
CLIENT_SECRET = 'FKC5BPFX5A3DLZJNDBC413YNIWXGWJI53FFEE3CFXO4MHENT' # your Foursquare Secret
VERSION = '20190531'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

address = 'San Francisco, SF'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)




Your credentails:
CLIENT_ID: ZNIGMWEGB3B2PRX1KKCUDR52MK2HBXMHHHXZUKK2D1KJDOIS
CLIENT_SECRET:FKC5BPFX5A3DLZJNDBC413YNIWXGWJI53FFEE3CFXO4MHENT
37.7792808 -122.4192363


In order to retrieve all venues around each neighborhood we create function which will get the top venues for each neighborhood from Foursquare and add the to a new dataset called sf_coffe_shops.


In [102]:
def getNearbyBars(names, latitudes, longitudes, radius=6000):
    bars_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        bars_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_bars = pd.DataFrame([item for bars_list in bars_list for item in bars_list])
    nearby_bars.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_bars)


sf_coffe_shops = getNearbyBars(names = sf_data['Neighborhood'],
                                   latitudes = sf_data['Latitude'],
                                   longitudes = sf_data['Longitude']
                                  )                                  
sf_coffe_shops.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Louise M. Davies Symphony Hall,37.777976,-122.420157,Concert Hall
1,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Herbst Theater,37.779548,-122.420953,Concert Hall
2,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Asian Art Museum,37.780178,-122.416505,Art Museum
3,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,War Memorial Opera House,37.778601,-122.420816,Opera House
4,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,SFJazz Center,37.776350,-122.421539,Jazz Club


We plot all venues returned from Foursquare in category Coffee Shop on a map to visualize the distribution.  


In [107]:
sf_coffe_shops = sf_bars.loc[sf_bars['Venue Category'] == 'Coffee Shop']
sf_coffe_shops.shape

(27, 7)

The returned results are 27. Further we will cluster only the neighborhoods in this dataset in which have a coffee shops. 


In [108]:

map_sf = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, venue, neighborhood in zip(sf_coffe_shops['Venue Latitude'], sf_coffe_shops['Venue Longitude'], sf_coffe_shops['Venue'], sf_coffe_shops['Neighborhood']):
    label = '{}, {}'.format(neighborhood, venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sf)  
    
map_sf


We are using k-means algorithm for clustering the neighborhoods based on the data in the dataset we have created. We choose to have 5 clusters and set the k to 5. After this we add a new column to our dataset with the information for the cluster labels for each neighborhood. 


In [111]:
k=5
sf_clstering = sf_coffe_shops.drop(['Neighborhood','Venue Latitude','Venue Longitude','Venue','Venue Category'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(sf_clstering)
kmeans.labels_
sf_coffe_shops.insert(0, 'Cluster Labels', kmeans.labels_)
sf_coffe_shops

,Cluster Labels,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
8,1,Hayes Valley/Tenderloin/North of Market,37.780,-122.420,Blue Bottle Coffee,37.776286,-122.416867,Coffee Shop
9,1,Hayes Valley/Tenderloin/North of Market,37.780,-122.420,Blue Bottle Coffee,37.776430,-122.423224,Coffee Shop
10,1,Hayes Valley/Tenderloin/North of Market,37.780,-122.420,Ritual Coffee Roasters,37.776476,-122.424281,Coffee Shop
29,1,Hayes Valley/Tenderloin/North of Market,37.780,-122.420,Sightglass Coffee,37.777001,-122.408519,Coffee Shop
34,1,South of Market,37.780,-122.410,Sightglass Coffee,37.777001,-122.408519,Coffee Shop
40,1,South of Market,37.780,-122.410,Blue Bottle Coffee,37.776286,-122.416867,Coffee Shop
79,1,Potrero Hill,37.770,-122.390,Blue Bottle Coffee,37.782497,-122.392982,Coffee Shop
85,1,Potrero Hill,37.770,-122.390,Sightglass Coffee,37.777001,-122.408519,Coffee Shop
95,1,Chinatown,37.791,-122.409,The Coffee Movement,37.794687,-122.410299,Coffee Shop
96,1,Chinatown,37.791,-122.409,Blue Bottle Coffee,37.792771,-122.404833,Coffee Shop



We plot the neighborhoods on a map as each cluster is in different color.


In [112]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sf_coffe_shops['Neighborhood Latitude'], sf_coffe_shops['Neighborhood Longitude'], sf_coffe_shops['Neighborhood'], sf_coffe_shops['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We check how many coffee shops are grouped to each cluster and choose the cluster with the second or third biggest number of coffee shops in it. Because the neighborhoods in the cluster  with high number of coffee shops will have big competition. The neighborhoods in the cluster with the second or the third biggest number will have existing coffee shops but not as many as the other cluster. 

We show the distribution of the bars in each cluster.


In [113]:
sf_coffe_shops['Cluster Labels'].value_counts()

1    12
3     5
2     4
4     3
0     3
Name: Cluster Labels, dtype: int64

We choose the cluster number 3 and inspect the neighborhoods in it. 

The preposition is the new coffee shop to be open in St. Francis Wood Neighborhood.


In [116]:
final_result = sf_coffe_shops.loc[sf_coffe_shops['Cluster Labels'] == 3]
final_result

,Cluster Labels,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
189,3,Ingelside-Excelsior/Crocker-Amazon,37.72,-122.44,Pinhole Coffee,37.739591,-122.418991,Coffee Shop
206,3,Ingelside-Excelsior/Crocker-Amazon,37.72,-122.44,Four Barrel Coffee,37.728967,-122.403840,Coffee Shop
207,3,Ingelside-Excelsior/Crocker-Amazon,37.72,-122.44,Philz Coffee,37.751143,-122.438361,Coffee Shop
464,3,St. Francis Wood/Miraloma/West Portal,37.73,-122.46,Philz Coffee,37.751143,-122.438361,Coffee Shop
479,3,St. Francis Wood/Miraloma/West Portal,37.73,-122.46,Pinhole Coffee,37.739591,-122.418991,Coffee Shop


It must be noted that the performance of the above algorithm and methodology highly depends of the returned data from Foursquare. The project is showing the methodology of the data science approach for solving a problem using Foresquare location data.


